In [271]:
n = 4
G = graphs.CompleteGraph(n)
B = Matroid(G)

In [272]:
L = B.lattice_of_flats()

In [273]:
frozenChains = list(L.chains())

In [274]:
maxChains = [[list(chain) for chain in p][1:] for p in frozenChains if len(p) == B.rank() + 1]
len(maxChains)

18

The list frozenChains contains all the chains in the lattice $L$ as frozensets. We define the set maxChains that contains all the maximal chains in the lattice $L$.

The structure of fy-monomials is such that for any strictly increasing chain of nonempty flats $F_1\subset F_2\subset\ldots\subset F_k$ in $L$ we have monomials of the form:
$$\{x_{F_{1}}^{m_{1}}\cdots x_{F_{k}}^{m_{k}}\mid 0\leq m_i\leq \text{rk}(F_i)-\text{rk}(F_{i-1})-1\}$$

with $F_0$ defined as the empty flat. It turns out to be enough to work only with the maximal chains. Then, for any maximal chain $F_1\subset F_2\subset\ldots\subset F_n$ and a monomial corresponding to the chain, we can assign a list $[i_1,\cdots, i_n]$ with $i_j$ as the degree of $F_j$. Here, we will call such a list as weight of the chain. Since all maximal chains have same cardinality, it is enough to generate the set of possible weights on just one chain. We do this as follows.

For a list $[ 1, 2, ..., n]$, we need to assign a weight (a positive integer) to each integer as follows:

    1. The weight at 1 is always 0.

    2. If k is the weight at j>1, then the weight at j-1, j-2,...,j-k must be zero.

The weights on [1,2,3,4] are

[[0,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1], [0,1,0,1], [0,0,2,0], [0,0,0,2], [0,0,0,3]]

In [275]:
def generate_weights(rank):
    weights = set() 
    for i in range(1, rank):
        for j in range(rank):
            weight = [0] * rank
            
            if i >= j:
                weight[i] = j
                weights.add(tuple(weight))

            if rank - (i+1) > 1:
                y = generate_weights(rank - (i + 1))  # Recursion to get the complete list of weights...
                for x in y:
                    temp_weight = weight.copy()
                    weights.add(tuple(temp_weight[:i+1] + x)) 

    return [list(w) for w in weights]

rank = n-1
all_weights = generate_weights(rank)

Now, for each chain and a weight list, we form monomials as follows:

chain: ```[[a], [a, b], [a, b, c, d, e, f], [a,b,c,d,e,f,g,h,i,j]]```

weight: ```[0,1,0,1]```

monomial: ```{([a],0), ([a, b],1), ([a, b, c, d, e, f],0), ([a,b,c,d,e,f,g,h,i,j],1)}```

This is a degree two monomial since the weight sum to 2.

In [276]:
def generate_fyMonomials(maxChains, weights):
    fy_monomials = [set() for _ in range(len(weights[0]))]
    for chain in maxChains:
        for weight in weights:
            degree = sum(weight)
            fy_monomial = []
            for i, segment in enumerate(chain):
                if weight[i] > 0:
                    fy_monomial.append((tuple(segment), weight[i]))
            fy_monomials[degree].add(frozenset(fy_monomial))
    return fy_monomials

fy_monomials = generate_fyMonomials(maxChains, all_weights)

In [277]:
dimensions = [0] * rank
for i in range(rank):
    dimensions[i] = len(fy_monomials[i])
dimensions

[1, 8, 1]

Now we perform the action of the group $S_n$ on $B$. We only need the characters of the irreducible representations of $S_n$ on each representative of the conjugacy classes of $S_n$. So, we first compute the conjugacy classes as follows:

In [278]:
def generate_conjugacyClasses(n):
    def subdivide_set(partition_set):
        partitions = []
        elements = list(range(n))
        for length in partition_set:
            partition = elements[:length]
            partitions.append(partition)
            for i in range(length):
                elements.pop(0)
        return partitions

    conjugacy_classes = Partitions(n).list()
    subdivided_partitions = [subdivide_set(part) for part in conjugacy_classes]

    irreducible_Representations = []
    for partition in subdivided_partitions:
        irr_representation = {}
        for x in partition:
            for i, element in enumerate(x):
                irr_representation[element] = x[(i + 1) % len(x)]
        irreducible_Representations.append(irr_representation)
        
    return conjugacy_classes, irreducible_Representations

conjugacy_Classes, irreducible_Representations = generate_conjugacyClasses(n)
print(conjugacy_Classes)
print(irreducible_Representations)

[[4], [3, 1], [2, 2], [2, 1, 1], [1, 1, 1, 1]]
[{0: 1, 1: 2, 2: 3, 3: 0}, {0: 1, 1: 2, 2: 0, 3: 3}, {0: 1, 1: 0, 2: 3, 3: 2}, {0: 1, 1: 0, 2: 2, 3: 3}, {0: 0, 1: 1, 2: 2, 3: 3}]


In [279]:
mutable_fyMonomials = []
for graded_piece in fy_monomials:
    mutable_graded_piece = [list(monomial) for monomial in graded_piece]
    mutable_fyMonomials.append(mutable_graded_piece)

In [280]:
def apply_map(monomial, mapping):
    mapped_monomial = []
    for term in monomial:
        new_term = []
        for edge in term[0]:
            new_edge = tuple(sorted([mapping[int(x)] for x in edge]))
            new_term.append(new_edge)
        mapped_monomial.append(tuple((tuple(new_term), term[1])))
    return tuple(sorted(mapped_monomial))

def monomial_equality(monomial, mapped_monomial):
    #sorting the inner tuples, and outer tuples and so on and on
    
    sorted_monomial = [(sorted(
        [tuple(sorted(segment)) for segment in m[0]]), m[1]
                              ) for m in monomial]
    
    sorted_mapped_monomial = [(sorted(
        [tuple(sorted(segment)) for segment in m[0]]), m[1]
                              ) for m in mapped_monomial]
    sorted_monomial.sort()
    sorted_mapped_monomial.sort()

    for i in range(len(sorted_monomial)):
        if sorted_monomial[i] != sorted_mapped_monomial[i]:
            return False

    return True

def character(temp_fyMonomials, irr_representation):
    i = 0
    for monomial in temp_fyMonomials:
        mapped_monomial = tuple(sorted(apply_map(monomial, irr_representation)))
        if monomial_equality(monomial, mapped_monomial): i += 1
    return i

characters_list = []
for i in range(rank):
    characters_list.append([character(mutable_fyMonomials[i], mapping) for mapping in irreducible_Representations])
print(characters_list)

[[1, 1, 1, 1, 1], [2, 2, 4, 4, 8], [1, 1, 1, 1, 1]]


In [281]:
def representations(n):
    representations = []
    partitions = Partitions(n).list()
    for partition in partitions:
        representations.append(
            SymmetricGroupRepresentation(partition).to_character().values()
            )
    return representations

In [282]:
import numpy as np


def decomposition(characters_list, n):
    # solving the equation Ax = b.

    trace_matrix = np.matrix(representations(n), dtype = float)
    # print("Character Table : ", trace_matrix)
    decomposition = []
    A = np.array(trace_matrix.transpose())
    for v in characters_list:
        v.reverse()
        rhs_array = np.array(v, dtype=float)
        solution = np.linalg.solve(A, rhs_array)
        decomposition.append(list(np.array(np.round(solution), dtype = int)))

    return decomposition

print(decomposition(characters_list, n))


[[1, 0, 0, 0, 0], [3, 1, 1, 0, 0], [1, 0, 0, 0, 0]]
